In [25]:
import os
import sys
import pandas as pd
from torchvision import transforms
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from dataset import GarbageClsDataset

In [ ]:
from PIL import Image, ImageOps

def pad_to_square(image: Image.Image, fill_color=(0, 0, 0)) -> Image.Image:
    width, height = image.size
    max_dim = max(width, height)
    padded_image = ImageOps.pad(image, (max_dim, max_dim), color=fill_color, centering=(0.5, 0.5))
    return padded_image

In [27]:
dataset = GarbageClsDataset('../data/annotations.csv', '../data/imgs/', transform = transforms.Compose([
        pad_to_square, 
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ]))

In [28]:
i, _ = dataset.__getitem__(0)

In [29]:
i

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [30]:
import torch
from torch.utils.data import DataLoader

# Assicurati che il tuo dataset non abbia trasformazioni di normalizzazione applicate
# in questa fase. Dovrebbe restituire tensori grezzi (es. 0-255 o 0-1).
# Esempio: train_dataset = YourDataset(annotations_file, transform=transforms.ToTensor())

# Usa un DataLoader per efficienza
loader = DataLoader(dataset, batch_size=64, shuffle=False)

# Inizializza le somme
mean = torch.zeros(3)
std = torch.zeros(3)
n_images = 0

# Itera sul DataLoader
for images, _ in loader:
    # Calcola la dimensione del batch
    batch_samples = images.size(0)
    # Riformatta le immagini per calcolare le statistiche per canale
    # da (N, C, H, W) a (N, C, H*W)
    images = images.view(batch_samples, images.size(1), -1)
    # Calcola la media per canale per il batch corrente e aggiorna la media totale
    mean += images.mean(2).sum(0)
    # Calcola la deviazione standard per canale per il batch corrente e aggiorna
    std += images.std(2).sum(0)
    # Aggiorna il conteggio totale delle immagini
    n_images += batch_samples

# Calcola la media e la deviazione standard finali
mean /= n_images
std /= n_images

print(f"Media calcolata per canale (RGB): {mean}")
print(f"Deviazione standard calcolata per canale (RGB): {std}")

Media calcolata per canale (RGB): tensor([0.5581, 0.5410, 0.5185])
Deviazione standard calcolata per canale (RGB): tensor([0.3177, 0.3070, 0.3034])
